<a href="https://colab.research.google.com/github/avneesh-nolkha/EVA5/blob/master/Session12/EVA5_Assignment12TinyImageNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from PIL import Image
import time
import cv2
import numpy as np
import requests
import zipfile
from tqdm import notebook
from io import StringIO,BytesIO
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset, random_split

In [2]:
!pip install git+https://github.com/smitasasindran/eva5utils.git

  Cloning https://github.com/smitasasindran/eva5utils.git to /tmp/pip-req-build-519rq76_
  Running command git clone -q https://github.com/smitasasindran/eva5utils.git /tmp/pip-req-build-519rq76_
     |████████████████████████████████| 634kB 4.6MB/s 
  Created wheel for eva5utils: filename=eva5utils-0.1.0-cp36-none-any.whl size=31737 sha256=c9cde7d538c9b6d930e734e6073196b1b12bb48cdc71f55219c398560329f355
  Stored in directory: /tmp/pip-ephem-wheel-cache-c8qc0ad3/wheels/c6/bb/ac/e3952b213bfdb25cb23715782e1a6b4b4b5cb10e42a2b80aa9
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654021 sha256=d5f24f322dfe55277549bb40811dbef380df6e00eaffefa6d1d847b63581f1cd
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built eva5utils imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [3]:
from eva5utils.model import ModifiedResNet18
from eva5utils.train import train_loop
from eva5utils.test import test_loop
from eva5utils.utils import plot_samples
from eva5utils.utils.helpers import show_model_summary, DEVICE, IS_CUDA

In [4]:
# Constants, put in config
epochs = 50
cuda_batch_size=256
cpu_batch_size = 4
num_workers = 4

model = ModifiedResNet18()
show_model_summary(model.to(DEVICE), (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,728
       BatchNorm2d-2           [-1, 64, 64, 64]             128
            Conv2d-3           [-1, 64, 64, 64]          36,864
       BatchNorm2d-4           [-1, 64, 64, 64]             128
            Conv2d-5           [-1, 64, 64, 64]          36,864
       BatchNorm2d-6           [-1, 64, 64, 64]             128
        BasicBlock-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
           Conv2d-10           [-1, 64, 64, 64]          36,864
      BatchNorm2d-11           [-1, 64, 64, 64]             128
       BasicBlock-12           [-1, 64, 64, 64]               0
           Conv2d-13          [-1, 128, 32, 32]          73,728
      BatchNorm2d-14          [-1, 128,

In [5]:
torch.cuda.is_available()

True

In [6]:
def download_images(url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"):
  if (os.path.isdir("data/datasets/tiny-imagenet-200")):
    print ('Images already downloaded...')
    return
  r = requests.get(url, stream=True)
  print ('Downloading TinyImageNet Data' )
  zip_ref = zipfile.ZipFile(BytesIO(r.content))
  for file in notebook.tqdm(iterable=zip_ref.namelist(), total=len(zip_ref.namelist())):
      zip_ref.extract(member = file,path='data/datasets/tiny-imagenet-200/')
  zip_ref.close()
  
def class_names(url = "data/datasets/tiny-imagenet-200/tiny-imagenet-200/wnids.txt"):
  f = open(url, "r")
  classes = []
  for line in f:
    classes.append(line.strip())
  return classes 

In [7]:
download_images()
classes = class_names()
print(len(classes))


200


In [8]:
class TinyImageNet(Dataset):
    def __init__(self, classes, url = "data/datasets/tiny-imagenet-200/tiny-imagenet-200", transform=None):
        self.data = []
        self.target = []
        self.classes = classes
        self.url = url
        self.transform = transform
        
        wnids = open(f"{url}/wnids.txt", "r")
        
        for wclass in notebook.tqdm(wnids,desc='Loading Train Folder', total = 200):
          wclass = wclass.strip()
          for i in os.listdir(url+'/train/'+wclass+'/images/'):
            img = Image.open(url+"/train/"+wclass+"/images/"+i)
            npimg = np.asarray(img)
            #There are some 1 channel image (Binary image) so reshape to 3 channel
            if(len(npimg.shape) ==2):
               npimg = np.repeat(npimg[:, :, np.newaxis], 3, axis=2)#repeat channel 1 data for other channel also
            self.data.append(npimg)  
            self.target.append(self.classes.index(wclass))

        val_file = open(f"{url}/val/val_annotations.txt", "r")
        for i in notebook.tqdm(val_file,desc='Loading Test Folder',total =10000 ):
          split_img, split_class = i.strip().split("\t")[:2]
          img = Image.open(f"{url}/val/images/{split_img}")
          npimg = np.asarray(img)
          #There are some 1 channel image (Binary image) so reshape to 3 channel
          if(len(npimg.shape) ==2):
            npimg = np.repeat(npimg[:, :, np.newaxis], 3, axis=2)#repeat channel 1 data for other channel also
          self.data.append(npimg)  
          self.target.append(self.classes.index(split_class))
            
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx]
        target = self.target[idx]
        img = data     
        if self.transform:
          data = self.transform(data)
        return data,target

In [9]:
class DatasetFromSubset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

In [10]:

from albumentations.pytorch import transforms as P
from albumentations.augmentations import transforms as A
from albumentations.core import composition as C

def model12_train_transforms():
  transform = C.Compose([
    A.PadIfNeeded(min_height=70, min_width=70, border_mode=cv2.BORDER_CONSTANT,
         value=0.5),
    A.RandomCrop(height=64, width=64),
    A.HorizontalFlip(p=0.5),
    A.Cutout(num_holes=1, max_h_size=32, max_w_size=32, p=1),
    P.ToTensor(dict (mean=(0.4802, 0.4481, 0.3975), std=(0.2302, 0.2265, 0.2262)))
    ])
  return lambda img: transform(image = np.array(img))["image"]

def model12_test_transforms():
  transform = C.Compose([
    P.ToTensor(dict (mean=(0.4802, 0.4481, 0.3975), std=(0.2302, 0.2265, 0.2262)))
    ])
  return lambda img: transform(image = np.array(img))["image"]

#    #P.ToTensor(dict (mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)))

#transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
# import torchvision.transforms as transforms
# def model_transforms():
#   transform = transforms.Compose([
#          transforms.RandomCrop(30, padding = 2),
#          transforms.RandomHorizontalFlip(),
#          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#          transforms.ToTensor()
#       ])
#   return transform

In [11]:
dataset = TinyImageNet(classes)
print('Dataset length: ',len(dataset))
train_data, test_data = random_split(dataset, [77000, 33000])


print('Trainset length: ',len(train_data))
print('Testset length: ',len(test_data))


Dataset length:  110000
Trainset length:  77000
Testset length:  33000


In [12]:
# Apply transformations on train and test data separately
train_data = DatasetFromSubset(train_data, model12_train_transforms())
test_data = DatasetFromSubset(test_data, model12_test_transforms())

In [13]:
dataloader_args = dict(shuffle=True, batch_size=cuda_batch_size, num_workers=num_workers, pin_memory=True) if IS_CUDA else dict(shuffle=True, batch_size=cpu_batch_size)

trainloader = torch.utils.data.DataLoader(train_data, **dataloader_args)
testloader = torch.utils.data.DataLoader(test_data, **dataloader_args)


In [14]:
img, label=train_data[0]
# img, label=trainloader.next()
# # img, label = iter(trainloader).next()
print(img.shape)


torch.Size([3, 64, 64])


In [15]:
# for i, batch in enumerate(trainloader):
#   img, label = batch
#   print(img.shape)
#   break

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1.45E-02 , momentum=0.9)         #3.85E-02, 2.18E-02
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=.2)
train_loss, train_acc = train_loop(epochs, trainloader, model, DEVICE, optimizer, criterion, scheduler, True)

[1,   301] loss: 1376.780
[2,   301] loss: 1155.626
[3,   301] loss: 1036.936
[4,   301] loss: 953.417
[5,   301] loss: 892.675
[6,   301] loss: 842.013
[7,   301] loss: 798.283
[8,   301] loss: 762.087
[9,   301] loss: 728.678
[10,   301] loss: 695.712
[11,   301] loss: 669.196
[12,   301] loss: 644.461
[13,   301] loss: 623.608
[14,   301] loss: 599.484
[15,   301] loss: 581.176
[16,   301] loss: 557.336
[17,   301] loss: 538.668
[18,   301] loss: 519.651
[19,   301] loss: 501.370
[20,   301] loss: 483.306
[21,   301] loss: 467.143
[22,   301] loss: 449.128
[23,   301] loss: 434.726
[24,   301] loss: 416.660
[25,   301] loss: 401.852
[26,   301] loss: 387.655
[27,   301] loss: 369.622
[28,   301] loss: 357.268
[29,   301] loss: 344.274
[30,   301] loss: 326.800
[31,   301] loss: 314.434
[32,   301] loss: 299.262
[33,   301] loss: 286.695
[34,   301] loss: 273.154
[35,   301] loss: 262.220
[36,   301] loss: 250.996
[37,   301] loss: 235.905
[38,   301] loss: 225.502
[39,   301] loss: 

In [17]:
test_loss, test_acc = test_loop(testloader, model, DEVICE, criterion)

Accuracy of the network on the 10000 test images: 50 %, Test loss: 20930.18455696106
